In [22]:
import os

In [31]:
os.chdir('D:\learnbay\DL PROjECTS\Kidney-Disease-Classification-MLflow-DVC')


<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\hp\AppData\Local\Temp\ipykernel_10768\1089411076.py:1: SyntaxWarning: invalid escape sequence '\l'
  os.chdir('D:\learnbay\DL PROjECTS\Kidney-Disease-Classification-MLflow-DVC')


In [32]:
os.environ["MLFROM_TRACKING_URI"] =  "https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sip4818"
os.environ["MLFLOW_TRACKING_PASSWORD"]="iamSahil@07"

In [33]:
# import dagshub
# dagshub.init(repo_owner='Sip4818', repo_name='Kidney-Disease-Classification-MLflow-DVC', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [34]:
# put into git bash
# export MLFLOW_TRACKING_URI ="https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow"
# export MLFLOW_TRACKING_USERNAME="Sip4818"
# export MLFLOW_TRACKING_PASSWORD="iamSahil@07"

In [35]:
import tensorflow as tf

In [36]:
import os
# os.chdir('../')

In [37]:
%pwd

'D:\\learnbay\\DL PROjECTS\\Kidney-Disease-Classification-MLflow-DVC'

In [38]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [40]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [41]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [42]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [60]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from pathlib import Path
import tensorflow as tf
import json

class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        
        # Save metrics to local JSON file (optional)
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        with open("scores.json", "w") as f:
            json.dump(scores, f, indent=4)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })





In [61]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


c:\Users\hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.3158 - loss: 22.5951
🏃 View run receptive-crow-583 at: https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow/#/experiments/0/runs/731399684bb5413a9e5fbbe078d119bd
🧪 View experiment at: https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow/#/experiments/0


In [51]:
os.environ["MLFROM_TRACKING_URI"] =  "https://dagshub.com/Sip4818/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sip4818"
os.environ["MLFLOW_TRACKING_PASSWORD"]="iamSahil@07"